In [ ]:
model_list = [
    "Tower-Babel/Babel-9B-Chat",
    "google/gemma-3-12b-it",
    "meta-llama/Llama-2-7b-chat-hf",
]

language_list = [
    "English",
    "German",
    "Korean",
]

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

version_colors = {
    "": "#1f77b4",  # Blue
    "v2": "#ff7f0e",  # Orange
    "v3": "#165a16",  # Green
    "v4": "#d62728"   # Red
}

version_linestyles = {
    "": ":",
    "v2": "--",
    "v3": "-.",
    "v4": ":"
}


model_colors = {
    "Babel-9B-Chat": "#66c2a5",
    "gemma-3-12b-it": "#fc8d62",
    "Llama-2-7b-chat-hf": "#e78ac3"
}

# Define colors for each language
language_colors = {
    "English": "#1f77b4",
    "Korean": "#ff7f0e",
    "German": "#2ca02c"
}

# Define line styles for each model
model_styles = {
    "Babel-9B-Chat": "-",
    "gemma-3-12b-it": "--",
    "Llama-2-7b-chat-hf": ":"
}

languague_styles = {
    "English": "-",
    "Korean": "--",
    "German": ":"
}

type_colors = {
    "v1": "#557fb1",
    "v2": "#ee6e44",
}

In [ ]:
# load dataset
import pandas as pd
import glob
import os

# Set your directory path here
directory_path = "/home/hyujang/multilingual-inner-lexicon/output/RQ1/ComponentAnalysis/attention_weights2"

# Find all CSV files ending with "_2token.csv"
# csv_files = glob.glob(os.path.join(directory_path, "*_2token.csv"))
csv_files = glob.glob(os.path.join(directory_path, "*token.csv"))


# Load and concatenate all DataFrames
# df_list = [pd.read_csv(file) for file in csv_files]
min_length = 10000  # Set your minimum length here
for file in csv_files:
    df = pd.read_csv(file)
    print(f"File: {file}, Length: {df.shape[0]}, max_context_length: {df['context'].apply(len).max()}")
    if df.shape[0] < min_length:   
        min_length = df.shape[0]

print(f"Minimum length across all files: {min_length}")

# filter the 
df_dict = {}
for file in csv_files:
    df = pd.read_csv(file)
    df = df.sample(n=min_length, random_state=2025)  # Sample min_length rows
    df_dict[file.split('/')[-1]] = df

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd
import glob
import os

# Define the directory path
directory_path = "/home/hyujang/multilingual-inner-lexicon/output/RQ1/ComponentAnalysis/attention_weights2"

# Create subplots with independent columns
fig, axes = plt.subplots(
    len(language_list),
    len(model_list),
    figsize=(15, 10),
    sharex='col',   # ✅ share only within columns
    sharey=True     # still share y across all subplots
)

# Ensure axes is 2D even if lists are length 1
if len(language_list) == 1:
    axes = axes.reshape(1, -1)
if len(model_list) == 1:
    axes = axes.reshape(-1, 1)

# Iterate over each model and language pair
for col_idx, model in enumerate(model_list):
    for row_idx, language in enumerate(language_list):
        ax = axes[row_idx, col_idx]

        # Find the corresponding files for _2token and _single-token
        file_2token = glob.glob(os.path.join(directory_path, f"{model.split('/')[-1]}_{language}_2token.csv"))
        file_single_token = glob.glob(os.path.join(directory_path, f"{model.split('/')[-1]}_{language}_1token.csv"))

        if not file_2token or not file_single_token:
            ax.set_title(f"{language}\nFile not found", fontsize=8)
            ax.axis('off')
            continue

        # Load the DataFrames
        df_2token = pd.read_csv(file_2token[0])
        df_single_token = pd.read_csv(file_single_token[0])

        # Extract layer-wise attention columns
        layer_columns = [col for col in df_2token.columns if col.startswith("layer_")]

        # Compute the average attention scores for each layer
        avg_attention_2token = df_2token[layer_columns].mean()
        avg_attention_single_token = df_single_token[layer_columns].mean()

        # Plot the attention scores
        ax.plot(
            range(1, len(avg_attention_2token) + 1),
            avg_attention_2token,
            marker='o',
            linestyle='--',
            # color='b',
            color="#557fb1",
            label=f'two-token words (n={len(df_2token)})'
        )
        ax.plot(
            range(1, len(avg_attention_single_token) + 1),
            avg_attention_single_token,
            marker='o',
            linestyle='--',
            color='#ee6e44',
            label=f'single-token words (n={len(df_single_token)})'
        )

        # Customize the subplot
        if row_idx == 0:
            ax.set_title(f"{model.split('/')[-1]}", fontsize=22)
        if col_idx == 0:
            ax.set_ylabel(f"{language}\nAvg. Attention Weight", fontsize=16)
        
        ax.set_xlabel("Layer", fontsize=16)
        ax.tick_params(axis='both', which='major', labelsize=14)
        ax.legend(fontsize=14)
        ax.grid(True)

# Adjust layout
plt.tight_layout()
plt.show()


In [ ]:
import pandas as pd

# Create a list to store the results
results = []

# Iterate over each dataset
for dataset_name, df in df_dict.items():
    # Extract layer-wise attention columns
    layer_columns = [col for col in df.columns if col.startswith("layer_")]

    # Ensure the second layer exists
    if "layer_2_attn" not in layer_columns:
        print(f"Second layer not found in {dataset_name}. Skipping.")
        continue

    # Get the second layer's attention weight
    second_layer_attn = df["layer_2_attn"]

    # Calculate the average difference with other layers
    for col in layer_columns:
        if col != "layer_2_attn":
            avg_diff = (df[col] - second_layer_attn).mean()
            results.append({
                "Dataset": dataset_name,
                "Layer": col,
                "Average Difference": avg_diff
            })

# Convert results to a DataFrame
results_df = pd.DataFrame(results)

# Display the results
print(results_df)